In [1]:
using POMDPs
using Distributions: Normal
using Random
import POMDPs: initialstate_distribution, actions, gen, discount, isterminal
Random.seed!(1);
using POMDPs, POMDPModels, POMDPSimulators, ARDESPOT, POMDPModelTools,POMDPPolicies
using ParticleFilters

In [2]:
struct goal_location
    x:: Int64
    y:: Int64
end

struct pedestrian_state
    x:: Int64
    y:: Int64
    goal:: goal_location
end

struct cart_state
    x:: Int64
    y:: Int64
    theta:: Int64
    v:: Int64
end    

struct golfcart_observations
    observed_human_positions:: Array{goal_location}
end

struct SP_POMDP_state
    cart:: cart_state
    pedestrians:: Array{pedestrian_state}
    pedestrian_goals:: Array{goal_location}
    path_covered_index:: Int64
end

struct human_goal_probability
    distribution::Array{Float64}
end

In [3]:
mutable struct Speed_Planner_POMDP <: POMDPs.POMDP{SP_POMDP_state,Int,golfcart_observations}
    discount_factor::Float64
    step_size::Int64
    collision_threshold::Float64
    collision_reward::Int64
    goal_reward::Int64
    max_cart_speed::Int64
    cart_goal_position::goal_location
    starting_cart_state::cart_state
    starting_human_states::Array{pedestrian_state}
    fixed_goal_locations::Array{goal_location}
    human_goals_prob_distribution::Array{human_goal_probability}
    astar_path::Array{Tuple{Int64,Int64}}
    start_path_index::Int64
end

In [37]:
function POMDPs.gen(m::Speed_Planner_POMDP, s, a, rng)
    
    # transition model
    
    function calculate_theta(current_state, previous_state)
        theta = 0
        x_diff = current_state[1] - previous_state[1]
        y_diff = current_state[2] - previous_state[2]
        if x_diff != 0
            if x_diff < 0
                theta = 90
            else
                theta = 270
            end
        end
        if y_diff != 0
            if y_diff < 0
                theta = 0
            else
                theta = 180
            end
        end
        return theta
    end
    
    function update_human_state(human, human_goals, rng)
        goal = human.goal
        human_fixed_goals = copy(human_goals)
        deleteat!(human_fixed_goals, findall(x -> x==goal, human_fixed_goals)[1])
        rand_num = rand(rng)
        function move_human_towards_goal(human, goal)
            temp_human_x = human.x
            temp_human_y = human.y
            if temp_human_x < goal.x
                temp_human_x = temp_human_x + 1
            elseif temp_human_x > goal.x
                temp_human_x = temp_human_x - 1
            end

            if temp_human_y < goal.y
                temp_human_y = temp_human_y + 1
            elseif temp_human_y > goal.y
                temp_human_y = temp_human_y - 1
            end
            temp_human_x = clamp(temp_human_x,1,14)
            temp_human_y = clamp(temp_human_y,1,30)
            return pedestrian_state(temp_human_x, temp_human_y, goal), goal_location(temp_human_x, temp_human_y)
        end
        if rand_num <= 0.7
            # move human towards goal
            new_human,observed_location = move_human_towards_goal(human, goal)
        elseif rand_num > 0.7 && rand_num <= 0.8
            new_human,observed_location = move_human_towards_goal(human, human_fixed_goals[1])
        elseif rand_num > 0.8 && rand_num <= 0.9
            new_human,observed_location = move_human_towards_goal(human, human_fixed_goals[2])
        elseif rand_num > 0.9
            new_human,observed_location = move_human_towards_goal(human, human_fixed_goals[3])
        end
        return new_human,observed_location
    end

    new_pedestrians = pedestrian_state[]
    observed_positions = goal_location[]
    
    # action 0
    if a == 0
        # kart state +2 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v
        new_v = s.cart.v + a
        new_position = m.astar_path[clamp(s.path_covered_index + new_v,1,length(m.astar_path))]
        new_theta = calculate_theta(new_position, m.astar_path[clamp(s.path_covered_index + new_v - 1,1,length(m.astar_path))])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
        
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human,observed_location = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
            push!(observed_positions, observed_location)
        end
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
    
    # action 1
    elseif a == 1
        # kart state +3 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v +1
        new_v = s.cart.v + a % 5
        new_position = m.astar_path[clamp(s.path_covered_index + new_v,1,length(m.astar_path))]
        new_theta = calculate_theta(new_position, m.astar_path[clamp(s.path_covered_index + new_v - 1,1,length(m.astar_path))])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
            
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human,observed_location = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
            push!(observed_positions, observed_location)
        end
        
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
        
    # action -1
    elseif a == -1
        # kart state +1 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v -1
        new_v = s.cart.v + a
        if new_v < 0
            new_v = 0
        end
        new_position = m.astar_path[clamp(s.path_covered_index + new_v,1,length(m.astar_path))]
        new_theta = calculate_theta(new_position, m.astar_path[clamp(s.path_covered_index + new_v - 1,1,length(m.astar_path))])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
        
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human,observed_location = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
            push!(observed_positions, observed_location)
        end
        
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
        
    end
    
    # update the state object
    sp = SP_POMDP_state(cart_new_state, new_pedestrians, s.pedestrian_goals, new_path_index)

    # observation model
    o = golfcart_observations(observed_positions)
    
    # reward model
    
    # collision reward
    function collision_reward(sp, coll_threshold)
        total_reward = 0
        cart_pose_x = sp.cart.x
        cart_pose_y = sp.cart.y
        for human in sp.pedestrians
            dist = ((human.x - cart_pose_x)^2 + (human.y - cart_pose_y)^2)^0.5
            if dist < coll_threshold
                total_reward = total_reward + m.collision_reward
            end
        end
        return total_reward
    end
    
    # goal reward
    function goal_reward(sp, s, goal_state_reward)
        total_reward = -1
        cart_new_pose_x = sp.cart.x
        cart_new_pose_y = sp.cart.y
        
        cart_goal = m.cart_goal_position
        new_dist = ((cart_goal.x - cart_new_pose_x)^2 + (cart_goal.y - cart_new_pose_y)^2)^0.5
        
        cart_old_pose_x = s.cart.x
        cart_old_pose_y = s.cart.y
        old_dist = ((cart_goal.x - cart_old_pose_x)^2 + (cart_goal.y - cart_old_pose_y)^2)^0.5
        
        if new_dist < old_dist && new_dist != 0
            total_reward = goal_state_reward/new_dist
        elseif new_dist == 0
            total_reward = goal_state_reward
        end
        return total_reward
    end
    
    # speed reward
    function speed_reward(sp, max_speed)
        return (sp.cart.v - max_speed)/max_speed
    end
    
    r = collision_reward(sp, m.collision_threshold) + goal_reward(sp, s, m.goal_reward) + speed_reward(sp, m.max_cart_speed)
    #@show("Action is ", a)
    # create and return a NamedTuple
    return (sp=sp, o=o, r=r)
    
end

In [38]:
#Discount and terminal state function

function isgoalstate(s,cart_goal)
    cart_x = s.cart.x
    cart_y = s.cart.y
    if(cart_goal.x == cart_x && cart_goal.y == cart_y)
        return true
    end
    for human in s.pedestrians
        if(cart_x == human.x && cart_y == human.y)
            #display("Collision")
            return true
        end
    end
    return false
end

discount(p::Speed_Planner_POMDP) = p.discount_factor
isterminal(p::Speed_Planner_POMDP, s::SP_POMDP_state) = isgoalstate(s,p.cart_goal_position);

In [39]:
#Action Space for the POMDP
actions(::Speed_Planner_POMDP) = [-1, 0, 1] # Decelerate Maintain Accelerate

actions (generic function with 17 methods)

In [40]:
#Initial state distribution for the POMDP

function initialstate_distribution(m::Speed_Planner_POMDP)
    initial_cart_state = m.starting_cart_state
    all_human_goal_locations = m.fixed_goal_locations
    initial_human_states = m.starting_human_states
    initial_path_start_index = m.start_path_index
    initial_human_goal_probability = m.human_goals_prob_distribution
    num_goals = length(all_human_goal_locations)
    
    all_256_possible_states = []
    all_256_probability_values = Float64[]
    
    for goal_human1_index in (1:num_goals)
        for goal_human2_index in (1:num_goals)
            for goal_human3_index in (1:num_goals)
                for goal_human4_index in (1:num_goals)
                    sampled_human1_state = pedestrian_state(initial_human_states[1].x,initial_human_states[1].y,all_human_goal_locations[goal_human1_index])
                    sampled_human2_state = pedestrian_state(initial_human_states[2].x,initial_human_states[2].y,all_human_goal_locations[goal_human2_index])
                    sampled_human3_state = pedestrian_state(initial_human_states[3].x,initial_human_states[3].y,all_human_goal_locations[goal_human3_index])
                    sampled_human4_state = pedestrian_state(initial_human_states[4].x,initial_human_states[4].y,all_human_goal_locations[goal_human4_index])
                    sampled_humans = [sampled_human1_state, sampled_human2_state, sampled_human3_state, sampled_human4_state]                    
                    generated_state = SP_POMDP_state(initial_cart_state,sampled_humans,all_human_goal_locations,initial_path_start_index)
                    push!(all_256_possible_states,generated_state)
                    
                    human1_prob = initial_human_goal_probability[1].distribution[goal_human1_index]
                    human2_prob = initial_human_goal_probability[2].distribution[goal_human2_index]
                    human3_prob = initial_human_goal_probability[3].distribution[goal_human3_index]
                    human4_prob = initial_human_goal_probability[4].distribution[goal_human4_index]
                    probability_for_generated_state =  human1_prob*human2_prob*human3_prob*human4_prob
                    push!(all_256_probability_values,probability_for_generated_state)
                end
            end
        end
    end
    d = SparseCat(all_256_possible_states, all_256_probability_values)
    #@show(eltype(d.probs))               
    return d
end

initialstate_distribution (generic function with 9 methods)

In [44]:
#Upper bound for DESPOT

function golf_cart_upper_bound(m, b)
    value_sum = 0.0
    function is_collision_state(s)
        is_collision_flag = false
        for human in s.pedestrians
            dist = ((human.x - s.cart.x)^2 + (human.y - s.cart.y)^2)^0.5
            if dist < m.collision_threshold
                is_collision_flag = true
            end
        end
        return is_collision_flag
    end
    function time_to_goal(s)
        curr_vel = m.max_cart_speed
        remaining_path_length = clamp(length(m.astar_path) - s.path_covered_index,1,100)
        time_needed_at_curr_vel = ceil(remaining_path_length/curr_vel)
        return time_needed_at_curr_vel
    end
    for (s, w) in weighted_particles(b)
        if(s.cart.x == 7 && s.cart.y==1)
            value_sum += w*m.goal_reward
        elseif (is_collision_state(s))
            value_sum += w*m.collision_reward*(-1)
        else
            value_sum += w*((discount(m)^time_to_goal(s))*m.goal_reward)
            #value_sum += w*m.goal_reward 
        end
    end
    #@show(value_sum)
    return (value_sum)/weight_sum(b)
end

golf_cart_upper_bound (generic function with 1 method)

In [45]:
function get_best_possible_action(cart_start_state_list,cart_goal_position,pedestrians_list,possible_goal_positions,
        initial_human_goal_distribution_list,given_astar_path)
    
    g1 =  goal_location(possible_goal_positions[1][1],possible_goal_positions[1][2])
    g2 =  goal_location(possible_goal_positions[2][1],possible_goal_positions[2][2])
    g3 =  goal_location(possible_goal_positions[3][1],possible_goal_positions[3][2])
    g4 =  goal_location(possible_goal_positions[4][1],possible_goal_positions[4][2])
    all_goals_list = [g1,g2,g3,g4]

    cart_start_state = cart_state(cart_start_state_list[1],cart_start_state_list[2],
        cart_start_state_list[3],cart_start_state_list[4])
    cart_goal = goal_location(cart_goal_position[1],cart_goal_position[2])

    ps1 = pedestrian_state(pedestrians_list[1][1],pedestrians_list[1][2], 
        goal_location(pedestrians_list[1][3][1],pedestrians_list[1][3][2]))
    ps2 = pedestrian_state(pedestrians_list[2][1],pedestrians_list[2][2], 
        goal_location(pedestrians_list[2][3][1],pedestrians_list[2][3][2]))
    ps3 = pedestrian_state(pedestrians_list[3][1],pedestrians_list[3][2], 
        goal_location(pedestrians_list[3][3][1],pedestrians_list[3][3][2]))
    ps4 = pedestrian_state(pedestrians_list[4][1],pedestrians_list[4][2],
        goal_location(pedestrians_list[4][3][1],pedestrians_list[4][3][2]))
    human_state_start_list = [ps1,ps2,ps3,ps4]

    h1_dis = human_goal_probability(initial_human_goal_distribution_list[1])
    h2_dis = human_goal_probability(initial_human_goal_distribution_list[2])
    h3_dis = human_goal_probability(initial_human_goal_distribution_list[3])
    h4_dis = human_goal_probability(initial_human_goal_distribution_list[4])
    human_dis_list = [h1_dis,h2_dis,h3_dis,h4_dis]

    robot_path = Tuple{Int64,Int64}[]
    
    for position in given_astar_path
        push!(robot_path,(position[1],position[2]))
    end
            
    golfcart_pomdp() = Speed_Planner_POMDP(0.9,1,2,-1000,100,5,cart_goal,cart_start_state,
        human_state_start_list,all_goals_list,human_dis_list,robot_path,1)    
    
    m = golfcart_pomdp()
    #solver = DESPOTSolver(bounds=(-20.0, 1000.0))
    #solver = DESPOTSolver(bounds=(DefaultPolicyLB(RandomSolver()), golf_cart_upper_bound))
    solver = DESPOTSolver(bounds=IndependentBounds(DefaultPolicyLB(RandomSolver()), 
            golf_cart_upper_bound, check_terminal=true))
    planner = solve(solver, m)
    b = initialstate_distribution(m);
    a = action(planner, b) 
    return a
    
end

get_best_possible_action (generic function with 1 method)

In [46]:
cart_start_state_list = [8,30,0,2]
cart_goal_position = [7,1]
pedestrians_list = [[4,3,[14,30]], [3,22,[14,1]], [11,22,[1,30]], [12,4,[1,1]]]
possible_goal_positions = [[1,1],[1,30],[14,30],[14,1]]
initial_human_goal_distribution_list = [[0.15,0.3,0.5,0.05], [0.1,0.3,0.1,0.5], 
    [0.5,0.25,0.1,0.15], [0.05,0.5,0.35,0.1]]
given_astar_path = [( 8, 30),( 7, 30),( 7, 29),( 7, 28),( 7, 27),( 7, 26),( 7, 25),
    ( 7, 24),( 7, 23),( 7, 22),( 6, 22),( 6, 21),( 5, 21),( 5, 20),( 5, 19),
             ( 5, 18),( 5, 17),( 5, 16),( 5, 15),( 5, 14),( 5, 13),( 5, 12),( 5, 11),
             ( 5, 10),( 5,  9),( 5,  8),( 5,  7),( 5,  6),
             ( 5,  5),( 5,  4),( 5,  3),( 5 , 2),( 5 , 1),( 6,  1),( 7,  1)];

action_to_be_taken = get_best_possible_action(cart_start_state_list,cart_goal_position,pedestrians_list,possible_goal_positions,
        initial_human_goal_distribution_list,given_astar_path);
@show(action_to_be_taken)

┌ Warning: L_0 (65.11495534064495) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (53.03432732390914) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (81.6445842073743) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (86.7203230953612) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (60.061086713519394) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (102.56960992876672) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (82.16639101607596) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/

┌ Warning: L_0 (102.9589288952587) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (76.2949813980471) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (50.833572972165456) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (81.10261333641391) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (88.93304542980404) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (91.27380748499463) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (98.36781779119899) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/

┌ Warning: L_0 (95.76146704919091) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (100.06867891650192) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (68.59745169641481) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (81.06455678481333) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (56.68959297188966) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (96.08700861750252) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (90.763180021495) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/t

┌ Warning: L_0 (93.47499966958813) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (125.77100363756897) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (78.42928940438776) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (77.0417572684599) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (109.36108922228532) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (66.69148391654828) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (55.004351206929485) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPO

┌ Warning: L_0 (109.68694367225025) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (84.21383390568766) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (95.48169118207042) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (141.3275687830228) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (102.24672679792657) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (152.80687090417547) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (128.32355389016703) > U_0 (47.82969000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDES

...

┌ Warning: L_0 (102.28658084481583) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (153.87845046257073) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (162.1791926615727) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (98.83339304511394) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (104.31927599643141) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (85.62285458887271) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (153.79172264679875) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDES

┌ Warning: L_0 (111.93953907105099) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (127.27690458579868) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (117.5394648678105) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (114.10363668980935) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (111.70308988678516) > U_0 (59.04900000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (117.4331766678105) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (110.7194202977759) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDES

action_to_be_taken = 1


┌ Warning: L_0 (108.80522934902413) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (130.96660854178802) > U_0 (59.04900000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (103.75751341972253) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (151.69805172838886) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (130.97734973985996) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (106.70683999547384) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/ARDESPOT/thdGA/src/bounds.jl:8
┌ Warning: L_0 (105.48184998921819) > U_0 (53.14410000000001)   |ϕ| = 1
└ @ ARDESPOT /home/himanshu/.julia/packages/AR

1

In [ ]:
# function get_best_possible_action(m::Speed_Planner_POMDP)
   
#     solver = DESPOTSolver(bounds=(-20.0, 1000.0))
#     #solver = DESPOTSolver(bounds=(DefaultPolicyLB(RandomSolver()), golf_cart_upper_bound))
#     planner = solve(solver, m);
#     b = initialstate_distribution(m);
#     a = action(planner, b) 
#     return a
    
# end
#get_best_possible_action(golfcart_pomdp())